In [ ]:
## **From Integer Linear Programming (ILP)**

In [1]:
!pip install qiskit-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 643.4/643.4 kB 4.3 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 15.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.7/22.7 MB 5.4 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 12.2 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.28.240-py3-none-any.whl size=682834 sha256=80cefef9dee57902a7f9992b2ecc2ddc250f7a46ad09c4faa072965984f3b7a7
  Stored in directory: /Users/liljaco/Library/Caches/pip/wheels/6a/f5/10/6df138f

## **Imports**

In [2]:
from qiskit_optimization.problems import QuadraticProgram

## **1. From Integer Linear Programming (ILP) to Quadratic Unconstrained Binary Optimization (QUBO)**

### **Define the ILP formulation of the BPP**

In [67]:
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo



def QuadraticProgram_BPP_ILP(object_sizes, container_capacity):
    num_objects = len(object_sizes)  # Number of objects
    max_containers = num_objects  # Maximum number of containers (can have up to num_objects containers)
    # Create the quadratic program
    qp = QuadraticProgram()

    # Binary variables for containers
    container_usage_variables = {}
    for container in range(max_containers):
        container_var = f"y_{container + 1}" # Variable to indicate if the container is in use
        container_usage_variables[container_var] = 1  # Coefficient for minimization
        qp.binary_var(container_var)

    # Objective function: minimize the number of containers used
    qp.minimize(linear=container_usage_variables)

    # Constraint 1: Each object must be assigned to exactly one container
    for obj in range(num_objects):
        object_assignment_vars = {}
        for container in range(max_containers):
            assignment_var = f"x_{obj+1}{container+1}" # Variable for object assignment to containers
            object_assignment_vars[assignment_var] = 1
            qp.binary_var(assignment_var)
        qp.linear_constraint(linear=object_assignment_vars, sense="==", rhs=1, name=f"object_{obj + 1}_assigned_constraint")

    # Constraint 2: The total size of objects in each container must not exceed its capacity
    for container in range(max_containers):
        container_load_vars = {}
        for obj in range(num_objects):
            assignment_var = f"x_{obj+1}{container+1}"
            container_load_vars[assignment_var] = object_sizes[obj]  # Size of the object
        
        container_var = f"y_{container + 1}" 
        container_load_vars[container_var] = -container_capacity  # Container capacity
        qp.linear_constraint(linear=container_load_vars, sense="<=", rhs=0, name=f"container_{container+1}_capacity_constraint")

    # Print the problem formulation
    # print(qp.prettyprint())
    return qp


# Problem parameters
object_sizes = [4, 8, 1, 4,]  # Sizes of the objects
container_capacity = 10  # Capacity of the containers


qp = QuadraticProgram_BPP_ILP(object_sizes,container_capacity)
conv = QuadraticProgramToQubo()
qubo_problem = conv.convert(qp)

# Print the QUBO problem
print(qubo_problem.prettyprint())


Problem name: 

Minimize
  5*container_1@int_slack@0^2
  + 20*container_1@int_slack@0*container_1@int_slack@1
  + 40*container_1@int_slack@0*container_1@int_slack@2
  + 30*container_1@int_slack@0*container_1@int_slack@3
  + 20*container_1@int_slack@1^2
  + 80*container_1@int_slack@1*container_1@int_slack@2
  + 60*container_1@int_slack@1*container_1@int_slack@3
  + 80*container_1@int_slack@2^2
  + 120*container_1@int_slack@2*container_1@int_slack@3
  + 45*container_1@int_slack@3^2 + 5*container_2@int_slack@0^2
  + 20*container_2@int_slack@0*container_2@int_slack@1
  + 40*container_2@int_slack@0*container_2@int_slack@2
  + 30*container_2@int_slack@0*container_2@int_slack@3
  + 20*container_2@int_slack@1^2
  + 80*container_2@int_slack@1*container_2@int_slack@2
  + 60*container_2@int_slack@1*container_2@int_slack@3
  + 80*container_2@int_slack@2^2
  + 120*container_2@int_slack@2*container_2@int_slack@3
  + 45*container_2@int_slack@3^2 + 5*container_3@int_slack@0^2
  + 20*container_3@int_sl

In [68]:
print(qp.prettyprint())

Problem name: 

Minimize
  y_1 + y_2 + y_3 + y_4

Subject to
  Linear constraints (8)
    x_11 + x_12 + x_13 + x_14 == 1  'object_1'
    x_21 + x_22 + x_23 + x_24 == 1  'object_2'
    x_31 + x_32 + x_33 + x_34 == 1  'object_3'
    x_41 + x_42 + x_43 + x_44 == 1  'object_4'
    4*x_11 + 8*x_21 + x_31 + 4*x_41 - 10*y_1 <= 0  'container_1'
    4*x_12 + 8*x_22 + x_32 + 4*x_42 - 10*y_2 <= 0  'container_2'
    4*x_13 + 8*x_23 + x_33 + 4*x_43 - 10*y_3 <= 0  'container_3'
    4*x_14 + 8*x_24 + x_34 + 4*x_44 - 10*y_4 <= 0  'container_4'

  Binary variables (20)
    y_1 y_2 y_3 y_4 x_11 x_12 x_13 x_14 x_21 x_22 x_23 x_24 x_31 x_32 x_33 x_34
    x_41 x_42 x_43 x_44

